In [12]:
from strategy.weekly_price import WeeklyPrice
from strategy.quarterly_financial_categorical import QuarterlyFinancialCategorical
from backtester.backtester import Backtester as b
from database.market import Market
from preprocessor.model_preprocessor import ModelPreprocessor as mp
from modeler.modeler import Modeler as m
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np
import math
import pickle

In [13]:
## Loading Constants
start = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
market = Market()
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
strat = WeeklyPrice()
qfc = QuarterlyFinancialCategorical()

In [14]:
categories = qfc.retrieve_data("portfolio_stock_category_sim")

In [15]:
categories.columns

Index(['_id', 'year', 'quarter', 'ticker', 'prediction'], dtype='object')

In [16]:
market.connect()
classification = market.retrieve_data("portfolio_dataset_week_classification")
regression = market.retrieve_data("portfolio_dataset_week_regression")
market.close()

In [17]:
gap = 5
week_gap = int(gap/5)
training_years = 1
quarterly_range = range(3,4)
yearly_range = range(2021,2022)

In [18]:
for year in tqdm(yearly_range):
    for quarter in tqdm(quarterly_range):
        quarter_categories = categories[(categories["year"] == year) & (categories["quarter"] == quarter)]
        for category in tqdm(quarter_categories["prediction"].unique()):
            try:
                group_tickers = quarter_categories[(quarter_categories["prediction"] == category)]
                prices = []
                if group_tickers.index.size > 0:
                    for ticker in group_tickers["ticker"].unique():
                        try:
                            ticker_class = classification.copy()[["year","quarter","week",ticker]].rename(columns={ticker:"y_class"})
                            ticker_regr = regression.copy()[["year","quarter","week",ticker]].rename(columns={ticker:str(0)})
                            test = ticker_regr.merge(ticker_class,on=["year","quarter","week"])
                            for i in range(1,14):
                                test[str(i)] = test[str(0)].shift(1)
                            test["y"] = test[str(0)].shift(-1)
                            test["y_class"] = test["y_class"].shift(-1)
                            f = test.dropna()
                            for col in ["year","quarter","week"]:
                                f[col] = [int(x) for x in f[col]]
                            f["date"] = [datetime(int(row[1]["year"]),int((row[1]["quarter"] * 3) - 2),1) for row in f.iterrows()]
                            f["ticker"] = ticker
                            prices.append(f)
                        except Exception as e:
                            print(str(e))
                            continue
                if len(prices) > 0:
                    p = pd.concat(prices)
                    training_data = p[(p["date"] < datetime(year,int((quarter*3)-2),1)) & (p["date"]>= datetime(year-training_years,int((quarter*3)-2),1))]
                    prediction_data = p[p["date"]==datetime(year,int((quarter*3)-2),1)]
                    X = training_data.drop(["quarter","year","y_class","y","date","ticker"],axis=1)
                    y = training_data[["y_class","y"]]
                    refined_data = {"X":X,"y":y}
                    regression_models = m.regression(refined_data.copy(),ranked=False,tf=False,xgb=True,sk=False,deep=False)
                    classification_models = m.classification(refined_data.copy(),tf=False,xgb=True,sk=False,deep=False,multioutput=False)
                    models = pd.DataFrame([regression_models,classification_models])
                    factors = refined_data["X"].columns
                    models["category"] = category
                    models["year"] = year
                    models["quarter"] = quarter
                    models["model"] = [pickle.dumps(x) for x in models["model"]]
                    strat.db.connect()
                    strat.db.store_data("portfolio_weekly_models",models)
                    strat.db.close()
            except Exception as e:
                print(str(e))
                continue

  0%|                                                                                                                                                   | 0/1 [00:00<?, ?it/s]

  0%|                                                                                                                                                  | 0/23 [00:00<?, ?it/s]

  9%|████████████                                                                                                                              | 2/23 [00:03<00:41,  1.98s/it]

 13%|██████████████████                                                                                                                        | 3/23 [00:09<01:03,  3.18s/it]

 17%|████████████████████████                                                                                                                  | 4/23 [00:17<01:26,  4.56s/it]

 22%|██████████████████████████████                                                                                     

In [19]:
categories["quarter"].max()

3